In [9]:
#Y=Diabetes
#X= Factors
import pandas as pd
import io
import numpy as np

from google.colab import files
data_to_load = files.upload()
print(data_to_load.keys())
df = pd.read_csv(io.BytesIO(data_to_load['diabetes2.csv']))

print(df)


Saving diabetes2.csv to diabetes2.csv
dict_keys(['diabetes2.csv'])
     Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0              6      148             72             35        0  33.6   
1              1       85             66             29        0  26.6   
2              8      183             64              0        0  23.3   
3              1       89             66             23       94  28.1   
4              0      137             40             35      168  43.1   
..           ...      ...            ...            ...      ...   ...   
763           10      101             76             48      180  32.9   
764            2      122             70             27        0  36.8   
765            5      121             72             23      112  26.2   
766            1      126             60              0        0  30.1   
767            1       93             70             31        0  30.4   

     DiabetesPedigreeFunction  Age  Outcome 

In [10]:

X = df.iloc[:,:-1]
y = df.iloc[:,-1]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X,y,test_size=0.25, random_state=0)

from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)



In [11]:
from sklearn.linear_model import LogisticRegression
classifier=LogisticRegression()
classifier.fit(X_train,y_train)



LogisticRegression()

In [12]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_curve, auc, precision_recall_curve
from sklearn import preprocessing
from sklearn.decomposition import PCA
import warnings
warnings.filterwarnings('ignore')

In [14]:
pipe=Pipeline([('standardization', preprocessing.StandardScaler()),
              ('pca', PCA()),
              ('lr', LogisticRegression(C=0.01, solver='lbfgs',max_iter=1000))])
param_grid={ 'pca__n_components': range(1, 200)}
grid= GridSearchCV(pipe, cv= 3, param_grid=param_grid,scoring='roc_auc',refit=True)
grid_fit = grid.fit(X_train,y_train)

In [16]:
y_pred = classifier.predict(X_test)
y_pred


array([1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1,
       1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0])

In [17]:

from sklearn.metrics import confusion_matrix, accuracy_score
print(confusion_matrix(y_test,y_pred))
print(accuracy_score(y_test,y_pred))



[[118  12]
 [ 26  36]]
0.8020833333333334


In [18]:
import numpy as np
from sklearn.linear_model import LinearRegression


In [20]:
from sklearn.linear_model import LassoCV, LassoLarsCV, LassoLarsIC
from sklearn import datasets
import time

model_bic = LassoLarsIC(criterion="bic", normalize=False)
#t1 = time.time()
model_bic.fit(X, y)
print(model_bic.coef_)

var = []
for i in range(len(model_bic.coef_)):
    if(model_bic.coef_[i] != 0):
        var.append(X.columns[i])

print(var)

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.1)



#selected variables
model_lasso = LassoLarsIC(criterion="bic", normalize=False)
model_lasso.fit(X_train[var], y_train)
Lasso_pred = model_lasso.predict(X_test[var])
print('\n')
print("Selected Variables MSE:",mean_squared_error(y_test,Lasso_pred))

#all variables
model_lassoAV = LassoLarsIC(criterion="bic", normalize=False)
model_lassoAV.fit(X_train, y_train)
Lasso_predAV = model_lassoAV.predict(X_test)
print("All Variables MSE:",mean_squared_error(y_test,Lasso_predAV))

#lassoCV
model_LCV = LassoCV(cv=20)
model_LCV.fit(X_train[var], y_train)
LCV_pred = model_LCV.predict(X_test[var])
print("LassoCV MSE:",mean_squared_error(y_test,LCV_pred))
print("\n")


[ 0.02059187  0.00592027 -0.00233188  0.00015452 -0.00018053  0.01324403
  0.14723744  0.00262139]
['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age']


Selected Variables MSE: 0.1427681221090455
All Variables MSE: 0.1427681221090455
LassoCV MSE: 0.14285074637738535




In [35]:
from sklearn import datasets
import statsmodels.api as sm
import statsmodels.formula.api as smf
import pandas as pd
import numpy as np
from itertools import cycle
from tqdm.auto import tqdm

df = pd.read_csv('diabetes2.csv')
#df = pd.get_dummies(data=df,columns=['Pregnancies', 'BMI', 'Glucose','Age'], drop_first=True, dtype=None)

#regression model
X = df.drop(['BMI'],axis=1)
y = df['BMI']
# Note the difference in argument order
model = sm.OLS(y,X).fit()
predictions = model.predict(X) # make the predictions by the model
# Print out the statistics
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                    BMI   R-squared (uncentered):                   0.946
Model:                            OLS   Adj. R-squared (uncentered):              0.945
Method:                 Least Squares   F-statistic:                              1664.
Date:                Thu, 14 Sep 2023   Prob (F-statistic):                        0.00
Time:                        23:45:32   Log-Likelihood:                         -2653.1
No. Observations:                 768   AIC:                                      5322.
Df Residuals:                     760   BIC:                                      5359.
Df Model:                           8                                                  
Covariance Type:            nonrobust                                                  
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Pregnancies                 -0.1135      0.100     -1.140      0.255      -0.309       0.082
Glucose                      0.1031      0.009     11.906      0.000       0.086       0.120
BloodPressure                0.1662      0.014     11.963      0.000       0.139       0.193
SkinThickness                0.2144      0.020     10.761      0.000       0.175       0.254
Insulin                     -0.0095      0.003     -3.368      0.001      -0.015      -0.004
DiabetesPedigreeFunction     2.6449      0.858      3.081      0.002       0.960       4.330
Age                          0.0738      0.029      2.560      0.011       0.017       0.130
Outcome                      1.3705      0.654      2.094      0.037       0.086       2.655
==============================================================================
Omnibus:                       45.210   Durbin-Watson:                   1.884
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              167.928
Skew:                           0.015   Prob(JB):                     3.43e-37
Kurtosis:                       5.291   Cond. No.                         569.
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""